In [1]:
%load_ext dotenv
%dotenv

In [2]:
import warnings
warnings.filterwarnings("ignore")

from model import chatmodel

In [3]:
import os
import oracledb

username=os.environ["DB_USER"]
password=os.environ["DB_PASSWORD"]
dsn=os.environ["DSN"]

try:
    conn23c = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!", conn23c.version)
except Exception as e:
    print("Connection failed!")

Connection successful! 23.4.1.24.6


In [4]:
from utils import use_oracle_db
retriever = use_oracle_db(conn23c)

기본 모델

In [5]:
chatmodel(retriever=retriever)

YOU: 난 사람 만나는 게 너무 싫어.. 집에만 있고 싶어 ㅠㅠ
AI: 당신은 INTP 유형의 특징 중 하나인 내향성을 가지고 있을 수 있습니다. INTP 유형은 주로 혼자 있는 시간을 선호하며, 자신의 아이디어와 생각에 집중하는 것을 좋아합니다. 사회적인 모임이나 사람들과의 대화에 대한 피로감을 느낄 수 있으며, 에너지를 충전하기 위해 혼자만의 시간이 필요합니다. 그러나 다양한 관심사와 창의성을 가지고 있는 INTP 유형은 자신의 관심 분야에 대해 깊이 파고들고 새로운 아이디어를 탐구하는 것을 즐깁니다.


활용 예시

In [5]:
add_to_system=""
add_history=[
    ("너, 내 MBTI가 뭔지 알아? 나 ISTJ야!",
     "좋은데요? 저 계획적인 사람 좋아해요!"),
]
chatmodel(add_to_system, add_history, retriever=retriever)

YOU: 넌 mbti가 뭐야??
AI: 저는 AI이기 때문에 MBTI 유형이 없어요. 하지만 여러분의 MBTI 유형에 대해 이야기하는 것은 재미있어요!
YOU: 알겠어. 내 친구는 isfp야. 내가 특별히 조심해야 할 언행이 있을까??
AI: 당신의 친구는 INFP(내향, 직관, 감정, 판단)입니다. 이 유형은 상상력이 풍부하고, 예술적이며, 인간 중심적인 경향이 있습니다. 당신의 친구를 위해 조심해야 할 언행은 다음과 같을 수 있습니다:

1. 비판이나 부정적인 발언: INFP는 비판에 민감하게 반응할 수 있습니다. 건설적인 비판도 조심스럽게 전달해야 합니다.
2. 과도한 세부사항: INFP는 큰 그림을 보는 것을 선호하므로, 세부사항에 너무 집중하는 것은 혼란스러울 수 있습니다.
3. 감정적인 솔직함: INFP는 자신의 감정을 솔직하게 표현하는 것을 좋아하지만, 때로는 다른 사람들의 감정을 상처주지 않도록 조심해야 합니다.
4. 개인적인 공간 존중: INFP는 내향적인 성격으로, 혼자 있는 시간을 필요로 합니다. 그들의 개인적인 공간을 존중해야 합니다.

INFP는 민감하고 예민한 면이 있지만, 창의적이고 따뜻한 마음을 가지고 있습니다. 그들의 감정을 이해하고 존중하는 것이 중요합니다.
YOU: 아니 isfp라고. infp가 아니라.
AI: 죄송합니다, 실수를 했네요. 당신의 친구는 ISFP(내향, 감각, 감정, 인식)입니다. 이 유형은 예술적이고, 감각적이며, 온화한 성격을 가지고 있습니다. 당신의 친구를 위해 조심해야 할 언행은 다음과 같을 수 있습니다:

1. 비판이나 부정적인 발언: ISFP는 비판에 민감하게 반응할 수 있습니다. 건설적인 비판도 조심스럽게 전달해야 합니다.
2. 과도한 세부사항: ISFP는 큰 그림을 보는 것을 선호하므로, 세부사항에 너무 집중하는 것은 혼란스러울 수 있습니다.
3. 감정적인 솔직함: ISFP는 자신의 감정을 솔직하게 표현하는 것을 좋아하지만, 때로는 다른 사람들의 감정을 상처주지 않도록 조심해야 합니다.
4. 개인

간략한 MBTI 검사

In [ ]:
add_to_system="""
    MBTI 검사에 대해서 알아? 16가지 성격유형, 그거야.
    공식 MBTI 검사는 비쌀 뿐만 아니라, 질문이 많아서 지루해하는 사람이 많아.
    MBTI에 관한 많은 자료를 출테니까, 나의 MBTI를 판단할 수 있는 질문 몇 개를 물어봐.
    내가 한 대답을 바탕으로, 네가 추정한 나의 MBTI를 말해줘.
    내가 대답하기 쉽도록 한 턴에 한 가지 질문만 해줘. 질문은 최대 10개를 넘지 않았으면 좋겠어.
    """
add_history=[]
chatmodel(add_to_system=add_to_system, add_history=add_history, retriever=retriever)